In [ ]:
!pip install openai


In [ ]:
from openai import OpenAI
import os


os.environ["GROQ_API_KEY"] = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"


client = OpenAI(
    api_key=os.environ["GROQ_API_KEY"],
    base_url="https://api.groq.com/openai/v1"
)

In [ ]:
conversation = [
    {"role": "user", "content": "Hello!"},
    {"role": "assistant", "content": "Hi! How can I help you today?"}
]


In [ ]:
def truncate_by_turns(conversation, n):
    return conversation[-n:]


In [ ]:
def truncate_by_length(conversation, max_chars=200):
    text = ""
    truncated = []
    for msg in reversed(conversation):
        if len(text) + len(msg["content"]) <= max_chars:
            truncated.insert(0, msg)
            text += msg["content"]
        else:
            break
    return truncated


In [ ]:
from openai import OpenAI
client = OpenAI()

def summarize_history(conversation):
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "Summarize the conversation briefly."},
            {"role": "user", "content": str(conversation)}
        ]
    )
    return response.choices[0].message.content


In [ ]:
def add_message(conversation, role, content, step, k=3):
    conversation.append({"role": role, "content": content})
    if step % k == 0:
        summary = summarize_history(conversation)
        conversation = [{"role": "system", "content": "Summary: " + summary}]
    return conversation


In [ ]:
schema = {
    "name": "extract_user_info",
    "description": "Extract user details from chat",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {"type": "string"},
            "email": {"type": "string"},
            "phone": {"type": "string"},
            "location": {"type": "string"},
            "age": {"type": "integer"}
        },
        "required": ["name", "email", "phone", "location", "age"]
    }
}


In [ ]:
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {"role": "user", "content": "Hi, I’m Logeshwari, 23 years old, from Coimbatore. My email is loges@example.com and my phone is 9876543210."}
    ],
    functions=[schema],
    function_call="auto"
)

result = response.choices[0].message
print(result)


ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=FunctionCall(arguments='{"age":23,"email":"loges@example.com","location":"Coimbatore","name":"Logeshwari","phone":"9876543210"}', name='extract_user_info'), tool_calls=None)


In [ ]:
import json
from jsonschema import validate

extracted = json.loads(result.function_call.arguments)
validate(instance=extracted, schema=schema["parameters"])
print("Validated JSON:", extracted)


Validated JSON: {'age': 23, 'email': 'loges@example.com', 'location': 'Coimbatore', 'name': 'Logeshwari', 'phone': '9876543210'}
